In [1]:
import tensorflow as tf

/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorf

In [49]:
folder = './'
file = 'model-500806.ckpt.meta'
sess = tf.Session()    
saver = tf.train.import_meta_graph(folder+file)
with sess.as_default() as sess:
    saver.restore(sess,tf.train.latest_checkpoint(folder))
output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(),
            ["action"]# The graph_def is used to retrieve the nodes  # The output node names are used to select the usefull nodes
        ) 
output_graph="./dogs-cats-model.pb"
with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(output_graph_def.SerializeToString())
sess.close()

INFO:tensorflow:Restoring parameters from ./model-500806.ckpt
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.


In [51]:
frozen_graph="./dogs-cats-model.pb"
with tf.gfile.GFile(frozen_graph, "rb") as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
 
with tf.Graph().as_default() as graph:
      tf.import_graph_def(graph_def,
                          input_map=None,
                          return_elements=None,
                          name="")

In [52]:
w1 = graph.get_tensor_by_name("vector_observation:0")

In [53]:
import numpy as np
feed_dict = {w1:np.random.rand(1,42)}

In [72]:
sess = tf.Session(graph= graph)
op = graph.get_tensor_by_name("action:0")
print(sess.run(op, feed_dict))

[[ 0.52686495 -0.27530786]]


In [66]:
with sess.as_default() as sess:
    print(sess.run("action:0", feed_dict))

[[-0.43747187  0.23002706]]


In [71]:
with sess.as_default() as sess:
    print(op.eval(feed_dict))

[[-0.45110226  0.5142297 ]]


In [76]:
op = graph.get_tensor_by_name("action:0")
print(sess.run(op, feed_dict))

[[0.14776514 0.1019797 ]]


In [75]:
print(feed_dict)

{<tf.Tensor 'vector_observation:0' shape=(?, 42) dtype=float32>: array([[0.35153244, 0.83004388, 0.52023986, 0.00613653, 0.09829448,
        0.09286162, 0.86073891, 0.91827396, 0.56635227, 0.56871414,
        0.26759645, 0.94488274, 0.50663586, 0.98145577, 0.77538203,
        0.9545396 , 0.48356375, 0.7419566 , 0.02212592, 0.62356835,
        0.30370853, 0.57289499, 0.60757296, 0.84101445, 0.34708852,
        0.94396402, 0.5665861 , 0.70916464, 0.65078087, 0.18058855,
        0.89537915, 0.52565546, 0.34320401, 0.18124406, 0.19237921,
        0.63016116, 0.33481808, 0.71768253, 0.96591474, 0.01590658,
        0.79192762, 0.66281535]])}


In [77]:
print(feed_dict)

{<tf.Tensor 'vector_observation:0' shape=(?, 42) dtype=float32>: array([[0.35153244, 0.83004388, 0.52023986, 0.00613653, 0.09829448,
        0.09286162, 0.86073891, 0.91827396, 0.56635227, 0.56871414,
        0.26759645, 0.94488274, 0.50663586, 0.98145577, 0.77538203,
        0.9545396 , 0.48356375, 0.7419566 , 0.02212592, 0.62356835,
        0.30370853, 0.57289499, 0.60757296, 0.84101445, 0.34708852,
        0.94396402, 0.5665861 , 0.70916464, 0.65078087, 0.18058855,
        0.89537915, 0.52565546, 0.34320401, 0.18124406, 0.19237921,
        0.63016116, 0.33481808, 0.71768253, 0.96591474, 0.01590658,
        0.79192762, 0.66281535]])}


In [78]:
def display_nodes(nodes):
    for i, node in enumerate(nodes):
        print('%d %s %s' % (i, node.name, node.op))
        [print(u'└─── %d ─ %s' % (i, n)) for i, n in enumerate(node.input)]

In [81]:
display_nodes(tf.GraphDef().node)

In [90]:
import time
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    t0 = time.time()
    for i in range(1,100):
        sess = tf.Session(graph= graph)
        op = graph.get_tensor_by_name("action:0")
        #print(sess.run(op, feed_dict))
    t1 = time.time()
    print(t1-t0)

0.021146535873413086


In [84]:
import tensorboard
%load_ext tensorboard

In [86]:
%tensorboard inline
%tensorboard --logdir './graphs'

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dty

invalid info file: '/tmp/.tensorboard-info/pid-10591.info'
Traceback (most recent call last):
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 316, in get_all
    info = _info_from_string(contents)
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 155, in _info_from_string
    raise ValueError("incompatible version: %r" % (json_value,))
ValueError: incompatible version: {'cache_key': 'eyJhcmd1bWVudHMiOlsiLS1sb2dkaXIiLCJsb2dkaXIiXSwiY29uZmlndXJlX2t3YXJncyI6e30sIndvcmtpbmdfZGlyZWN0b3J5IjoiL2hvbWUvc2FpL2hkZDEvbWwtbWFzdGVyL21sLWFnZW50cy9neW0tdW5pdHkifQ==', 'db': '', 'logdir': 'logdir', 'path_prefix': '', 'pid': 10591, 'port': 6008, 'start_time': 1595833491, 'version': '2.0.1'}
invalid info file: '/tmp/.tensorboard-info/pid-10110.info'
Traceback (most recent call last):
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 316, in get_all
    info = _info_from_

invalid info file: '/tmp/.tensorboard-info/pid-9874.info'
Traceback (most recent call last):
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 316, in get_all
    info = _info_from_string(contents)
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 155, in _info_from_string
    raise ValueError("incompatible version: %r" % (json_value,))
ValueError: incompatible version: {'cache_key': 'eyJhcmd1bWVudHMiOlsiLS1sb2dkaXIiLCJsb2dzIl0sImNvbmZpZ3VyZV9rd2FyZ3MiOnt9LCJ3b3JraW5nX2RpcmVjdG9yeSI6Ii9ob21lL3NhaS9oZGQxL21sLW1hc3Rlci9tbC1hZ2VudHMvZ3ltLXVuaXR5In0=', 'db': '', 'logdir': 'logs', 'path_prefix': '', 'pid': 9874, 'port': 6006, 'start_time': 1595832278, 'version': '2.0.1'}
invalid info file: '/tmp/.tensorboard-info/pid-10591.info'
Traceback (most recent call last):
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 316, in get_all
    info = _info_from_string(c

Reusing TensorBoard on port 6009 (pid 335), started 0:05:40 ago. (Use '!kill 335' to kill it.)

invalid info file: '/tmp/.tensorboard-info/pid-10591.info'
Traceback (most recent call last):
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 316, in get_all
    info = _info_from_string(contents)
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 155, in _info_from_string
    raise ValueError("incompatible version: %r" % (json_value,))
ValueError: incompatible version: {'cache_key': 'eyJhcmd1bWVudHMiOlsiLS1sb2dkaXIiLCJsb2dkaXIiXSwiY29uZmlndXJlX2t3YXJncyI6e30sIndvcmtpbmdfZGlyZWN0b3J5IjoiL2hvbWUvc2FpL2hkZDEvbWwtbWFzdGVyL21sLWFnZW50cy9neW0tdW5pdHkifQ==', 'db': '', 'logdir': 'logdir', 'path_prefix': '', 'pid': 10591, 'port': 6008, 'start_time': 1595833491, 'version': '2.0.1'}
invalid info file: '/tmp/.tensorboard-info/pid-10110.info'
Traceback (most recent call last):
  File "/home/sai/.conda/envs/cs231n/lib/python3.7/site-packages/tensorboard/manager.py", line 316, in get_all
    info = _info_from_

In [94]:
t0 = time.time()
for i in range(1,100):
    sess = tf.Session(graph= graph)
    op = graph.get_tensor_by_name("action:0")
    sess.run(op, feed_dict)
t1 = time.time()
100.0/(t1-t0)

75.64410961937936